In [44]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import keras as ks

In [45]:
%ls

 98bk02.h5         dTb02              index.png   'paramter estimation.ipynb'
 binary_data_02/   EmuBk0.2.h5        lvac02.png   x_HI02
 bk_norm02         hyperparam.ipynb   nbins02
 bk_un02           index              params02


In [46]:
N_bins = np.loadtxt('nbins02')
N_bins.shape

(1058, 550)

In [47]:
Bk = np.loadtxt('bk_norm02')
Bk.shape

(1058, 550)

In [6]:
params = np.loadtxt('params02')
params.shape

(1058, 3)

In [48]:
ind = np.loadtxt('index')
index = np.zeros(len(ind),dtype=int)
for i in range(len(ind)):
    index[i] = int(ind[i])
index

array([ 146,  833,  516,  204,  929,  223,  154,   42,   87,   10,  535,
        343,  525,  239,   37,  532,  966, 1043])

In [49]:
Bk_test = Bk[index]
params_test = params[index]
cov_inv_test = cov[index]
N_bins_test = N_bins[index]

In [50]:
cov_inv_test = np.zeros(shape=(len(Bk_test),550,550))
for i in range(len(Bk_test)):
    for j in range(550):
        cov_inv_test[i][j][j] = Bk_test[i][j]/np.sqrt(N_bins_test[i][j])

In [51]:
def Bk_model(params):
    
    model = ks.models.load_model('98bk02.h5')
    model = model.predict([[params]])
    model = model*100.
    
    return model

In [52]:
def lL(params,Bk,cov_inv):
    Nion,Rmfp,NoH = params
    if Nion<=0. or Rmfp<=0. or NoH<=0. :
        l=-np.inf
    else:

        model=Bk_model(params)
        DMU=Bk-model
        l=-0.5*np.dot(np.dot(DMU,cov_inv),DMU.T)
   
    return l[0][0]

In [53]:
ndim=3            #===== no. of parameters =======#

nwalkers=6      #========= no. of random walkers ========#

Nion = np.random.randint(20,200,6).T
Rmfp = np.random.randint(20,120,6).T
NoH = np.random.randint(50,1100,6).T
p = np.zeros(shape=(6,3))
p[:,0] = Nion
p[:,1] = Rmfp
p[:,2] = NoH


In [54]:
lL(p[0],Bk_test[0],cov_inv_test[0])

-61491495.9509039

In [43]:
from multiprocessing import Pool
import emcee
import time
nsteps = 10
with Pool() as pool:
    sampler=emcee.EnsembleSampler(nwalkers,ndim,lL,args=[Bk_test[0],cov_inv_test[0]])
    start = time.time()
    sampler.run_mcmc(p, nsteps, progress=True)
    end = time.time()
    multi_time = end - start
    print("Multiprocessing took {0:.1f} seconds".format(multi_time))



  0%|          | 0/10 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/home/ht/.local/lib/python3.6/site-packages/emcee/ensemble.py", line 545, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-39-e9ee108e504d>", line 11, in lL
    return l[0][0]
TypeError: 'float' object is not subscriptable
  0%|          | 0/10 [00:06<?, ?it/s]

emcee: Exception while calling your likelihood function:
  params: [  77.87159235   65.68972785 -210.28407651]
  args: [array([ 1.97045401e+01,  2.06078593e+01,  2.64638321e+01,  2.77957439e+01,
        2.35642960e+01,  2.93198530e+01,  2.54363947e+01,  2.66502327e+01,
        2.17456753e+01,  2.99353589e+01,  3.70847240e+01,  3.80188733e+01,
        3.18084400e+01,  3.67649859e+01,  1.80921127e+01,  1.73716388e+01,
        3.26595865e+01,  2.95522955e+01,  3.47005171e+01,  2.62425364e+01,
        2.60979628e+01,  1.61156194e+01,  1.86037628e+01,  2.05495787e+01,
        2.78359201e+01,  3.16528349e+01,  3.21574207e+01,  2.96310056e+01,
        5.43073668e+01,  4.70282972e+01,  6.71857717e+01,  8.65860506e+01,
        6.45859476e+01,  6.92684487e+01,  7.63956874e+01,  8.23574578e+01,
        6.38193021e+01,  3.48275999e+01,  5.37052577e+01,  4.84506968e+01,
        2.92199672e+01,  4.35256119e+01,  3.55987974e+01,  1.81555042e+01,
        5.37682661e+01,  5.39151993e+01,  6.12166546e+0

TypeError: 'float' object is not subscriptable